<a href="https://colab.research.google.com/github/anuragp-tm/AirfaceLens/blob/main/AirfaceLens_Object_and_Text_Detection_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt

width = 1028
height = 1028

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename
cap=take_photo()
print(cap)

<IPython.core.display.Javascript object>

photo.jpg


In [ ]:
from google.colab.patches import cv2_imshow


# capturing the video
  # camera = cv2.VideoCapture(0,cv2.CAP_DSHOW)
  # for i in range(1):
  # return_value, pic = camera.read()
  # cv2.imwrite('task1'+str(i)+'.png', pic)

#Resize to respect the input_shape
cap=cv2.imread('/content/photo.jpg')
inp = cv2.resize(cap, (width , height ))

#Convert img to RGB
rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)

# Converting to uint 8
rgb_tensor = tf.convert_to_tensor(rgb, dtype=tf.uint8)

#Add dims to rgb_tensor
rgb_tensor = tf.expand_dims(rgb_tensor , 0)

#we can load the model and the labels

import tensorflow_hub as hub
import pandas as pd

# Loading model directly from TensorFlow Hub
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite2/detection/1")

# Loading csv with labels of classes
labels = pd.read_csv('/content/labels.csv', sep=';', index_col='ID')
labels = labels['OBJECT (2017 REL.)']

#we can create the predictions and put in the image the boxes and labels found:
# Creating prediction
boxes, scores, classes, num_detections = detector(rgb_tensor)

# Processing outputs
pred_labels = classes.numpy().astype('int')[0]
pred_labels = [labels[i] for i in pred_labels]
pred_boxes = boxes.numpy()[0].astype('int')
pred_scores = scores.numpy()[0]

# Putting the boxes and labels on the image
for score, ( ymin,xmin,ymax,xmax), label in zip(pred_scores, pred_boxes, pred_labels):
    if score < 0.5:
        continue

    score_txt = f'{100 * round(score)}%'
    img_boxes = cv2.rectangle(rgb,(xmin, ymax),(xmax, ymin),(0,255,0),2)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img_boxes, label,(xmin, ymax-10), font, 1.5, (255,0,0), 2, cv2.LINE_AA)
    cv2.putText(img_boxes,score_txt,(xmax, ymax-10), font, 1.5, (255,0,0), 2, cv2.LINE_AA)
    cv2_imshow(img_boxes)

In [11]:
import easyocr
reader = easyocr.Reader(['en']) 
result = reader.readtext('/content/images/nokia.png',detail=0)
print(result)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


['Connecting People']


In [5]:
import easyocr
reader = easyocr.Reader(['en']) 
result = reader.readtext('/content/images/cocacola.jpeg',detail=0)
print(result)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


['"h0i4al t5', '(ca Gcla']


In [6]:
import easyocr
reader = easyocr.Reader(['en']) 
result = reader.readtext('/content/images/nescape.jpeg',detail=0)
print(result)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


['NEScAFE', 'cLA $', 'oor', 'E1']


In [7]:
import easyocr
reader = easyocr.Reader(['en']) 
result = reader.readtext('/content/images/crown.jpg',detail=0)
print(result)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


['CROWN', 'YOURLOGO HERE']


In [8]:
import easyocr
reader = easyocr.Reader(['en']) 
result = reader.readtext('/content/images/altmetric.png',detail=0)
print(result)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


['Altmetric']


In [10]:
import easyocr
reader = easyocr.Reader(['en']) 
result = reader.readtext('/content/images/cocacola.png',detail=0)
print(result)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


['GcGola']


In [13]:
import easyocr
reader = easyocr.Reader(['en']) 
result = reader.readtext('/content/images/disney.png',detail=0)
print(result)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


['DiSNep']


In [21]:
import easyocr
reader = easyocr.Reader(['en']) 
result = reader.readtext('/content/images/quote.jpg',detail=0)
print(result)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


['IF YOU', 'DONT KNOW', 'WHERE YOU ARE', 'GOING,', 'ANY ROAD', 'WILL GET YOU', 'THERE.', 'Ruwsy €civll', 'Sayinglmages com']


In [22]:
import easyocr
reader = easyocr.Reader(['en']) 
result = reader.readtext('/content/images/banner_quote.jpg',detail=0)
print(result)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


['is', 'fresh', 'start;', 'Every', 'day']
